In [4]:
import numpy as np
import geopandas as gpd
from shapely.geometry import box
from sklearn.cluster import DBSCAN
from pyproj import Transformer
from pathlib import Path

import rasterio
from rasterio.merge import merge
from rasterio.shutil import copy
from rasterio.enums import Resampling

def get_raster_bbox(raster_path):
    """Extract the bounding box of a raster file in EPSG:3857 (meters)."""
    with rasterio.open(raster_path) as dataset:
        bounds = dataset.bounds  # (left, bottom, right, top)

        # Convert coordinates to meters (EPSG:4326 to EPSG:3857)
        transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
        min_x, min_y = transformer.transform(bounds.left, bounds.bottom)
        max_x, max_y = transformer.transform(bounds.right, bounds.top)

        return min_x, min_y, max_x, max_y, raster_path

def bbox_distance(bbox1, bbox2):
    """Compute the minimum distance between two bounding boxes."""
    min_x1, min_y1, max_x1, max_y1, _ = bbox1
    min_x2, min_y2, max_x2, max_y2, _ = bbox2

    # Compute horizontal and vertical distances
    dx = max(min_x2 - max_x1, min_x1 - max_x2, 0)  # Distance along x-axis
    dy = max(min_y2 - max_y1, min_y1 - max_y2, 0)  # Distance along y-axis

    return np.hypot(dx, dy)  # Euclidean distance

def group_rasters_by_bbox(raster_paths, distance_threshold=10):
    """Groups raster files based on bounding box proximity."""
    bboxes = [get_raster_bbox(path) for path in raster_paths]

    # Compute pairwise distance matrix
    distance_matrix = np.zeros((len(bboxes), len(bboxes)))
    for i in range(len(bboxes)):
        for j in range(len(bboxes)):
            if i != j:
                distance_matrix[i, j] = bbox_distance(bboxes[i], bboxes[j])

    # Use DBSCAN clustering
    clustering = DBSCAN(eps=distance_threshold, min_samples=1, metric="precomputed").fit(distance_matrix)

    # Organize rasters by cluster labels
    grouped_rasters = {}
    for bbox, label in zip(bboxes, clustering.labels_):
        if label not in grouped_rasters:
            grouped_rasters[label] = []
        grouped_rasters[label].append(bbox)

    return list(grouped_rasters.values())

def merge_rasters_to_cog(groups, output_folder="merged_rasters"):
    """Merge each group into a single COG file."""
    output_folder = Path(output_folder)
    output_folder.mkdir(exist_ok=True)

    for i, group in enumerate(groups):
        raster_paths = [bbox[4] for bbox in group]  # Extract file paths
        print(f"Generate {i}")

        # Open all rasters
        datasets = [rasterio.open(path) for path in raster_paths]

        # Merge the rasters
        mosaic, out_transform = merge(datasets, resampling=Resampling.bilinear)

        # Use metadata from first raster
        out_meta = datasets[0].meta.copy()
        out_meta.update({
            "driver": "COG",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_transform,
            "compress": "LZW",
            "BIGTIFF": "IF_SAFER"
        })

        merged_path = Path(output_folder, f"group_{i}.tif")

        # Save the merged raster
        with rasterio.open(merged_path, "w", **out_meta) as dest:
            dest.write(mosaic)

        # Convert to COG
        cog_path = Path(output_folder, f"group_{i}_cog.tif")
        copy(merged_path, cog_path, driver="COG", compress="LZW")

        merged_path.unlink()

        # Close datasets
        for dataset in datasets:
            dataset.close()


raster_files = sorted(list(Path("./cog_data").iterdir()))

groups = group_rasters_by_bbox(raster_files, distance_threshold=10)

cog_files = merge_rasters_to_cog(groups)


Generate 0
Generate 1
Generate 2
Generate 3
Generate 4
Generate 5
Generate 6
Generate 7
Generate 8
Generate 9
Generate 10
Generate 11
Generate 12
Generate 13
Generate 14
Generate 15
Generate 16
Generate 17
Generate 18
Generate 19
Generate 20
Generate 21
Generate 22
Generate 23
Generate 24
Generate 25
Generate 26
Generate 27
Generate 28
Generate 29
Generate 30
Generate 31
Generate 32
Generate 33
Generate 34


# Utiliser gdal avec docker

`docker run --rm -it --user 1000:1000 -v /home/bioeos/Documents/project_hub/tms-server/bathy:/app ghcr.io/osgeo/gdal:latest`

## Commande pour convertir les rasters de bathy en couleurs.

`gdaldem color-relief group_24_cog.tif color.txt group_24_color.tif -alpha`

`color.txt` contient :

```text
-9999   0   0   0   0
nan     0   0   0   0
0       255 0 0 255
-5      255 255 255 255   
-40      0 0 255 255
```

## Commande pour convertir les rasters en tuiles.

gdal2tiles --processes=12 -x -n -z 14-22 group_24_color.tif tiles_group_24_color